# Import Libraries

In [2]:
import csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [3]:
import os
import numpy as np
import contractions as ctc
import re
import pandas as pd

In [4]:
from tensorflow.keras import models,layers
import tensorflow.keras.backend as K
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

In [103]:
import tensorflow_hub as hub
from sklearn.metrics import classification_report,f1_score

# Load Preprocess data

In [29]:
df_imdb=pd.read_csv("IMDB Dataset.csv")

In [30]:
labels2={"positive":1,"negative":0}

In [31]:
df_imdb["lables"]=df_imdb["sentiment"].map(labels2)

In [32]:
## Putting space b/w Special Characters
def is_special(text):
    rem = ''
    for i in text: 
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
            rem=rem+i
            rem = rem + ' '
    return rem

In [33]:
## removing extra spsces from text
def rem_extra(text):
    rem=re.sub(' +', ' ',text)
    return str(rem).strip()

In [34]:
df_imdb.review=df_imdb.review.apply(is_special)

In [35]:
df_imdb.review=df_imdb.review.apply(rem_extra)

In [36]:
df_train=df_imdb.iloc[0:25000]
df_test=df_imdb.iloc[25000:]

In [37]:
df_train.head()

,review,sentiment,lables
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production . < br / > < br ...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there ' s a family where a little bo...,negative,0
4,"Petter Mattei ' s "" Love in the Time of Money ...",positive,1


# Load Pretrained GloVe

In [38]:
embeddings_index = dict()
addrs="../ADL1/embeddings/glove.6B.300d.txt"
f = open(addrs)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [39]:
x_train=np.zeros((len(df_train.review),300))
k=0;
for i in df_train.review:
    vec=np.zeros((300))
    c=1;
    for word in i.split(): 
        embedding_vector = embeddings_index.get(word)
        #print(embedding_vector.shape)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
        else:
            #print(word)
            pass
    x_train[k]=vec/c
    k+=1

In [40]:
x_test=np.zeros((len(df_test.review),300))
k=0;
for i in df_test.review:
    vec=np.zeros((300))
    c=0;
    for word in i.split(): 
        embedding_vector = embeddings_index.get(word)
        #print(embedding_vector.shape)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
        else:
            #print(word)
            pass
    x_test[k]=vec/c
    k+=1
    

# DAN+DNN

In [57]:
model1=Sequential()
model1.add(InputLayer((300)))
model1.add(Dense(1024))
model1.add(Dense(512))
model1.add(Dropout(0.2))
model1.add(Dense(256))
model1.add(Dropout(0.2))
model1.add(Dense(512))
model1.add(Dense(256,activation="relu"))
model1.add(Dense(128,activation="relu"))
model1.add(Dense(5,activation="softmax"))

In [59]:
model1.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [60]:
es=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')

In [61]:
model1.fit(x_train,df_train.lables.values,epochs=100,batch_size=32,validation_split=0.1,callbacks=[es])

Epoch 1/100
704/704 [==============================] - 4s 5ms/step - loss: 0.4994 - accuracy: 0.7575 - val_loss: 0.4147 - val_accuracy: 0.8140
Epoch 2/100
704/704 [==============================] - 4s 5ms/step - loss: 0.4206 - accuracy: 0.8135 - val_loss: 0.4015 - val_accuracy: 0.8264
Epoch 3/100
704/704 [==============================] - 3s 5ms/step - loss: 0.4078 - accuracy: 0.8208 - val_loss: 0.4173 - val_accuracy: 0.8076
Epoch 4/100
704/704 [==============================] - 3s 5ms/step - loss: 0.4102 - accuracy: 0.8224 - val_loss: 0.4043 - val_accuracy: 0.8200
Epoch 5/100
704/704 [==============================] - 3s 5ms/step - loss: 0.3990 - accuracy: 0.8239 - val_loss: 0.4507 - val_accuracy: 0.7876
Epoch 6/100
704/704 [==============================] - 4s 5ms/step - loss: 0.3929 - accuracy: 0.8300 - val_loss: 0.4078 - val_accuracy: 0.8176
Epoch 7/100
704/704 [==============================] - 4s 5ms/step - loss: 0.3906 - accuracy: 0.8288 - val_loss: 0.5167 - val_accuracy: 0.7780

In [62]:
model1.evaluate(x_test,df_test.lables.values)

782/782 [==============================] - 1s 2ms/step - loss: 0.3909 - accuracy: 0.8324


[0.39091718196868896, 0.8324000239372253]

In [98]:
y_pred=np.argmax(model1.predict(x_test),axis=1)

In [99]:
pr=classification_report(df_test.lables.values,y_pred)

In [100]:
print(pr)

              precision    recall  f1-score   support

           0       0.80      0.89      0.84     12474
           1       0.88      0.77      0.82     12526

    accuracy                           0.83     25000
   macro avg       0.84      0.83      0.83     25000
weighted avg       0.84      0.83      0.83     25000



In [104]:
f1_score(df_test.lables.values,y_pred)

0.8224275300898458

# Load Pretrained Fast-text

In [63]:
embeddings_index = dict()
addrs="../ADL1/embeddings/wiki-news-300d-1M-subword.vec"
f = open(addrs)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 999995 word vectors.


In [67]:
x_train_f=np.zeros((len(df_train.review),300))
k=0;
for i in df_train.review:
    vec=np.zeros((300))
    c=1;
    for word in i.split(): 
        embedding_vector = embeddings_index.get(word)
        #print(embedding_vector.shape)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
        else:
            #print(word)
            pass
    x_train_f[k]=vec/c
    k+=1

In [68]:
x_test_f=np.zeros((len(df_test.review),300))
k=0;
for i in df_test.review:
    vec=np.zeros((300))
    c=1;
    for word in i.split(): 
        embedding_vector = embeddings_index.get(word)
        #print(embedding_vector.shape)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
        else:
            #print(word)
            pass
    x_test_f[k]=vec/c
    k+=1
    

# DAN +DNN

In [73]:
model2=Sequential()
model2.add(InputLayer((300)))
model2.add(Dense(1024))
model2.add(Dense(512))
model2.add(Dropout(0.2))
model2.add(Dense(256))
model2.add(Dropout(0.2))
model2.add(Dense(512))
model2.add(Dense(256,activation="relu"))
model2.add(Dense(128,activation="relu"))
model2.add(Dense(5,activation="softmax"))

In [74]:
model2.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [75]:
es=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')

In [76]:
model2.fit(x_train_f,df_train.lables.values,epochs=100,batch_size=32,validation_split=0.1,callbacks=[es])

Epoch 1/100
704/704 [==============================] - 4s 5ms/step - loss: 0.5855 - accuracy: 0.6884 - val_loss: 0.4996 - val_accuracy: 0.7668
Epoch 2/100
704/704 [==============================] - 3s 5ms/step - loss: 0.4623 - accuracy: 0.7882 - val_loss: 0.4702 - val_accuracy: 0.7960
Epoch 3/100
704/704 [==============================] - 3s 5ms/step - loss: 0.4425 - accuracy: 0.8020 - val_loss: 0.5090 - val_accuracy: 0.7680
Epoch 4/100
704/704 [==============================] - 3s 5ms/step - loss: 0.4290 - accuracy: 0.8090 - val_loss: 0.4305 - val_accuracy: 0.8056
Epoch 5/100
704/704 [==============================] - 3s 5ms/step - loss: 0.4229 - accuracy: 0.8115 - val_loss: 0.4421 - val_accuracy: 0.8016
Epoch 6/100
704/704 [==============================] - 3s 5ms/step - loss: 0.4175 - accuracy: 0.8156 - val_loss: 0.4294 - val_accuracy: 0.8136
Epoch 7/100
704/704 [==============================] - 4s 5ms/step - loss: 0.4106 - accuracy: 0.8200 - val_loss: 0.4204 - val_accuracy: 0.8072

In [78]:
model2.evaluate(x_test_f,df_test.lables.values)

782/782 [==============================] - 1s 2ms/step - loss: 0.3750 - accuracy: 0.8366


[0.37495118379592896, 0.8366400003433228]

In [105]:
y_pred=np.argmax(model2.predict(x_test_f),axis=1)

In [106]:
pr=classification_report(df_test.lables.values,y_pred)

In [107]:
print(pr)

              precision    recall  f1-score   support

           0       0.87      0.79      0.83     12474
           1       0.81      0.89      0.84     12526

    accuracy                           0.84     25000
   macro avg       0.84      0.84      0.84     25000
weighted avg       0.84      0.84      0.84     25000



In [108]:
f1_score(df_test.lables.values,y_pred)

0.8447620495666718

# Pretrained word2vec

In [80]:
# create a weight matrix for words in training docs
from gensim import models

w2v = models.KeyedVectors.load_word2vec_format('../ADL1/embeddings/GoogleNews-vectors-negative300.bin', binary=True)


In [85]:
x_train_w2v=np.zeros((len(df_train.review),300))
k=0;
for i in df_train.review:
    vec=np.zeros((300))
    c=1;
    for word in i.split(): 
        try:
            vec+=w2v[word]
            c+=1
        except:
            #print(word)
            pass
    x_train_w2v[k]=vec/c
    k+=1

In [87]:
x_test_w2v=np.zeros((len(df_test.review),300))
k=0;
for i in df_test.review:
    vec=np.zeros((300))
    c=1;
    for word in i.split(): 
        try:
            vec+=w2v[word]
            c+=1
        except:
            #print(word)
            pass
    x_test_w2v[k]=vec/c
    k+=1
    

In [89]:
model3=Sequential()
model3.add(InputLayer((300)))
model3.add(Dense(1024))
model3.add(Dense(512))
model3.add(Dropout(0.2))
model3.add(Dense(256))
model3.add(Dropout(0.2))
model3.add(Dense(512))
model3.add(Dense(256,activation="relu"))
model3.add(Dense(128,activation="relu"))
model3.add(Dense(5,activation="softmax"))

In [90]:
model3.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [91]:
es=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')

In [92]:
model3.fit(x_train_w2v,df_train.lables.values,epochs=100,batch_size=32,validation_split=0.1,callbacks=[es])

Epoch 1/100
704/704 [==============================] - 5s 6ms/step - loss: 0.4330 - accuracy: 0.8005 - val_loss: 0.3870 - val_accuracy: 0.8380
Epoch 2/100
704/704 [==============================] - 4s 5ms/step - loss: 0.3762 - accuracy: 0.8404 - val_loss: 0.3664 - val_accuracy: 0.8376
Epoch 3/100
704/704 [==============================] - 3s 5ms/step - loss: 0.3674 - accuracy: 0.8466 - val_loss: 0.3944 - val_accuracy: 0.8188
Epoch 4/100
704/704 [==============================] - 4s 5ms/step - loss: 0.3672 - accuracy: 0.8469 - val_loss: 0.3797 - val_accuracy: 0.8312
Epoch 5/100
704/704 [==============================] - 4s 6ms/step - loss: 0.3580 - accuracy: 0.8493 - val_loss: 0.3635 - val_accuracy: 0.8464
Epoch 6/100
704/704 [==============================] - 4s 6ms/step - loss: 0.3562 - accuracy: 0.8500 - val_loss: 0.3668 - val_accuracy: 0.8420
Epoch 7/100
704/704 [==============================] - 4s 5ms/step - loss: 0.3526 - accuracy: 0.8520 - val_loss: 0.3714 - val_accuracy: 0.8476

In [94]:
model3.evaluate(x_test_w2v,df_test.lables.values)

782/782 [==============================] - 2s 2ms/step - loss: 0.3398 - accuracy: 0.8567


[0.3398231565952301, 0.8566799759864807]

In [109]:
y_pred=np.argmax(model3.predict(x_test_w2v),axis=1)

In [110]:
f1_score(df_test.lables.values,y_pred)

0.8592860228566941